In [68]:
import numpy as np
import json
from snmp_stream import *
import snmp_stream as snmp

oid = ObjectIdentity(ObjectIdentity([1, 2]))

json.loads(json.dumps(oid, cls=ObjectIdentityEncoder), cls=ObjectIdentityDecoder)

ObjectIdentity('.1.2')

In [2]:
response = walk(
    '127.0.0.1:1161',
    ('recorded/linux-full-walk', 'V2C'),
    [
        '1.3.6.1.2.1.1.1',
        #'1.3.6.1.2.1.2',
    ],
    req_id='abc',
    config={'retries': 1, 'timeout': 3}
)
print(np.array2string(response.results.reshape(response.results.size >> 3, 8)))
response
# snmpsimd.py --agent-udpv4-endpoint=127.0.0.1:1161 --process-user=root --process-group=root

('recorded/linux-full-walk', 'V2C')
[[  8   8   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0]
 [ 97  98  99   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [ 64   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  2   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [112   0   0   0   0   0   0   0]
 [195  86  80  94   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  4   0   0   0   0   0   0   0]
 [  8   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [ 64   0   0   0   0   0   0   0]
 [ 76 105 110 117 120  32  99 114]
 [ 97 121  32  50  46  54  46  50]
 [ 49  46  53  45 115 109 112  32]
 [ 35  50  32  83  77  80  32  84]
 [117 101  32  74 117 110  32  49]
 [ 57  32  49  52  58  53  56  58]
 [ 49  49  32  67  68  84  32  50]
 [ 48  48  55  32 1

SnmpResponse(type=SUCCESSFUL, request=SnmpRequest(type=WALK_REQUEST, host='127.0.0.1:1161', communities=Community(string='recorded/linux-full-walk', version=V2C), oids=['.1.3.6.1.2.1.1.1'], ranges=None, req_id='abc', config=Config(retries=1, timeout=3, max_response_var_binds_per_pdu=10, max_async_sessions=10)), errors=[])

In [67]:
%time

def align(parser):
    def _align(word_size, octect_size, endianess, arr):
        _arr, val = parser(word_size, octect_size, endianess, arr)
        aligned = ((len(arr) - len(_arr)) + (word_size - 1)) & ~((word_size) - 1)
        return arr[aligned:], val
    return _align

def uword():
    def _word(word_size, octect_size, endianess, arr):
        word = arr[:word_size].view(endianess + 'u' + str(word_size))[0]
        return arr[word_size:], word
    return _word

def variable(parser):
    def _variable(word_size, octect_size, endianess, arr):
        arr, size = uword()(word_size, octect_size, endianess, arr)
        return arr[size:], parser(word_size, octect_size, endianess, arr[:size])[1]
    return _variable
        
def string():
    def _string(word_size, octect_size, endianess, arr):
        return np.array([], dtype=np.uint8), arr.tobytes().decode('utf-8')
    return _string

class SnmpResultParser:
    word_size: np.uint8
    octect_size: np.uint8
    endianess: np.uint8
    
    arr: np.ndarray
    metadata: str

    def __init__(self, results):
        self.word_size = results[0]
        self.octect_size = results[1]
        self.endianess = '<' if results[2] == 0 else '>'
        arr, self.metadata = align(variable(string()))(self.word_size, self.octect_size, self.endianess, results[self.word_size:])
        
parser = SnmpResultParser(response.results)
parser.metadata

In [67]:
{
    '1.3.6.1.2.1.1.1': (integer('sys_idx'), string('sys_descr'))
}

snmp.walk(
    '127.0.0.1:1161',
    ('recorded/linux-full-walk', 'V2C'),
    [
        '1.3.6.1.2.1.1.1',
        #'1.3.6.1.2.1.2',
    ],
    req_id='abc',
    config={'retries': 1, 'timeout': 3}
)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


'abc'

In [3]:

def decode(arr):
    word_size = arr[0]
    word_dtype = np.dtype('u'+str(word_size))
    octet_size = arr[1]
    octet_dtype = np.dtype('u'+str(octet_size))
    arr = arr[word_size:]

    def word(arr):
        return arr[word_size:], arr[:word_size].view(word_dtype)[0]

    def octet(arr):
        return arr[octet_size:], arr[:octet_size].view(octet_dtype)[0]

    def align(x):
        return (x + (word_size - 1)) & ~((word_size) - 1)

    def aligned(orig, res):
        arr, val = res
        return orig[align(len(orig) - len(arr)):], val

    def take_sized(arr):
        arr, size = word(arr)
        return arr[size:], arr[:size]
    
    def octet_string(arr):
        return arr.view(octet_dtype)
    
    def var_bind(arr):
        arr, timestamp = word(arr)
        arr, root_oid_index = word(arr)
        arr, value_type = word(arr)
        arr, index = aligned(arr, take_sized(arr))
        arr, value = aligned(arr, take_sized(arr))
        return timestamp, root_oid_index, value_type, index, value
        
    def _decode(arr):
        arr, metadata = aligned(arr, take_sized(arr))
        arr, root_oid_count = word(arr)
        root_oids = []
        
        for i in range(root_oid_count):
            arr, root_oid = aligned(arr, take_sized(arr))
            root_oids.append(octet_string(root_oid))
            
        resp_var_binds = []
        
        while arr.size != 0:
            arr, resp_var_bind = aligned(arr, take_sized(arr))
            resp_var_binds.append(var_bind(resp_var_bind))
        
        return metadata, root_oids, resp_var_binds, arr
    return _decode(arr)
    

decode(response.results)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


(array([97, 98, 99], dtype=uint8),
 [array([1, 3, 6, 1, 2, 1, 1, 1], dtype=uint64)],
 [(1582322325,
   0,
   4,
   array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
   array([ 76, 105, 110, 117, 120,  32,  99, 114,  97, 121,  32,  50,  46,
           54,  46,  50,  49,  46,  53,  45, 115, 109, 112,  32,  35,  50,
           32,  83,  77,  80,  32,  84, 117, 101,  32,  74, 117, 110,  32,
           49,  57,  32,  49,  52,  58,  53,  56,  58,  49,  49,  32,  67,
           68,  84,  32,  50,  48,  48,  55,  32, 105,  54,  56,  54],
         dtype=uint8))],
 array([], dtype=uint8))

In [4]:
np.set_printoptions(threshold=10000000000)
print(np.array2string(response.results.reshape(response.results.size >> 3, 8)))

[[  8   8   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0]
 [ 97  98  99   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [ 64   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  3   0   0   0   0   0   0   0]
 [  6   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  2   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [112   0   0   0   0   0   0   0]
 [149  82  80  94   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  4   0   0   0   0   0   0   0]
 [  8   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [ 64   0   0   0   0   0   0   0]
 [ 76 105 110 117 120  32  99 114]
 [ 97 121  32  50  46  54  46  50]
 [ 49  46  53  45 115 109 112  32]
 [ 35  50  32  83  77  80  32  84]
 [117 101  32  74 117 110  32  49]
 [ 57  32  49  52  58  53  56  58]
 [ 49  49  32  67  68  84  32  50]
 [ 48  48  55  32 105  54  56  54]]


In [29]:
response.results.size / 8

969.0

In [ ]:
np.dtype('u8').itemsize

In [ ]:
snmp.ObjectIdentity(oid='') <= [1,1,1]

In [ ]:
".1.1.1.1"[1:]

In [242]:
class A:
    pass

    
    

In [245]:
def print_a(a):
    if a is None:
        print('none')

    if a is not None:
        print(a)
        
print_a(1)
print_a(None)

1
none


In [335]:
a = [1, 2, 3]

In [337]:
a[:]

[3]